In [1]:
"""Calculate CRC checksum on the JIC bitstream

Altera bitstream uses CRC16 Modbus, as mentioned in P. Swierczynski:
"Security Analysis of the Bitstream Encryption Scheme of Altera FPGAs"

Requires crccheck package (`pip install crccheck`)
"""

import os
import sys

import numpy as np
from crccheck.crc import Crc16Modbus

sys.path.append("../extract_pin_addr")
from JicBitstream import JicBitstream, JicBitstreamZip

In [2]:
jic = JicBitstream("../../bitstreams/ref/factory.jic").jic_uint8

In [3]:
BLK_SIZE = 1188
CRC_SIZE = 2

# manually identified checksum locations
CHKSUM_LOCS = [
    0x13536,
    0x139DA,
    0x13E7E,
    0x14322,
    0x147C6,
]

# add more blocks at the beggining
for _ in range(0):
    CHKSUM_LOCS.insert(0, CHKSUM_LOCS[0] - BLK_SIZE)

# add more blocks at the end
for _ in range(10):
    CHKSUM_LOCS.append(CHKSUM_LOCS[-1] + BLK_SIZE)

for chksum_loc in CHKSUM_LOCS:
    print(
        chksum_loc,
        [hex(xi) for xi in jic[chksum_loc : chksum_loc + 2]],
        jic[chksum_loc + CRC_SIZE : chksum_loc + BLK_SIZE],
    )

print(np.diff(CHKSUM_LOCS))

79158 ['0x9d', '0x33'] [ 0  0  0 ... 64  0  0]
80346 ['0x7c', '0xec'] [0 0 0 ... 0 0 0]
81534 ['0xdc', '0xf4'] [0 0 0 ... 3 0 0]
82722 ['0x64', '0xc1'] [0 0 0 ... 0 0 0]
83910 ['0x36', '0x3a'] [0 0 0 ... 0 0 0]
85098 ['0x8f', '0x44'] [0 0 0 ... 0 0 0]
86286 ['0xc5', '0xb4'] [0 0 0 ... 0 0 0]
87474 ['0x93', '0xd'] [0 0 0 ... 0 0 0]
88662 ['0xb4', '0x83'] [0 0 0 ... 0 0 0]
89850 ['0x70', '0x3f'] [ 0  0  0 ... 64  0  0]
91038 ['0x71', '0x19'] [0 0 0 ... 0 0 0]
92226 ['0x77', '0xa4'] [0 0 0 ... 3 0 0]
93414 ['0x13', '0x35'] [0 0 0 ... 0 0 0]
94602 ['0xd0', '0x3b'] [ 0  0  0 ... 32  0  0]
95790 ['0xbb', '0x7e'] [ 0  0  0 ... 32  0  0]
[1188 1188 1188 1188 1188 1188 1188 1188 1188 1188 1188 1188 1188 1188]


In [4]:
BLOCK_LOCS = np.array(CHKSUM_LOCS) - BLK_SIZE + CRC_SIZE

for block_loc in BLOCK_LOCS:
    print(f"Calculating CRC for block 0x{block_loc:06x}", end="")
    data = jic[block_loc : block_loc + BLK_SIZE]
    checksum = Crc16Modbus.calc(data)
    print(f", checksum = {checksum:04x}")
    assert checksum == 0

Calculating CRC for block 0x013094, checksum = 0000
Calculating CRC for block 0x013538, checksum = 0000
Calculating CRC for block 0x0139dc, checksum = 0000
Calculating CRC for block 0x013e80, checksum = 0000
Calculating CRC for block 0x014324, checksum = 0000
Calculating CRC for block 0x0147c8, checksum = 0000
Calculating CRC for block 0x014c6c, checksum = 0000
Calculating CRC for block 0x015110, checksum = 0000
Calculating CRC for block 0x0155b4, checksum = 0000
Calculating CRC for block 0x015a58, checksum = 0000
Calculating CRC for block 0x015efc, checksum = 0000
Calculating CRC for block 0x0163a0, checksum = 0000
Calculating CRC for block 0x016844, checksum = 0000
Calculating CRC for block 0x016ce8, checksum = 0000
Calculating CRC for block 0x01718c, checksum = 0000
